In [66]:
import pandas as pd
import numpy  as np
from sklearn                import utils
from sklearn                import preprocessing
from sklearn.pipeline       import Pipeline
from sklearn.linear_model   import LinearRegression
from sklearn.preprocessing  import StandardScaler
from sklearn.metrics        import mean_squared_error
from sklearn.tree           import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor

### Extraccion

In [46]:
df_trai = pd.read_csv('../instrucciones/datosEntrenamiento.csv', names=['A', 'B', 'C', 'D', 'val'])
df_test = pd.read_csv('../instrucciones/datosPrueba.csv',        names=['A', 'B', 'C', 'D', 'val'])
df_vali = pd.read_csv('../instrucciones/datosValidacion.csv',    names=['A', 'B', 'C', 'D', 'val'])

### Exploracion

In [69]:
df_trai.corr()

,A,B,C,D,val
A,1.000000,0.843183,-0.503403,-0.543673,-0.948731
B,0.843183,1.000000,-0.410853,-0.309859,-0.869185
C,-0.503403,-0.410853,1.000000,0.096809,0.517436
D,-0.543673,-0.309859,0.096809,1.000000,0.389565
val,-0.948731,-0.869185,0.517436,0.389565,1.000000


In [27]:
df_trai.dtypes

A      float64
B      float64
C      float64
D      float64
val    float64
dtype: object

In [50]:
X_trai = df_trai.loc[:, ['A', 'B', 'C', 'D']].values
Y_trai = df_trai.loc[:, 'val'].values

In [47]:
X_test = df_test.loc[:, ['A', 'B', 'C', 'D']].values
Y_test = df_test.loc[:, 'val'].values

In [48]:
X_vali = df_test.loc[:, ['A', 'B', 'C', 'D']].values

### Preprocesamiento, entrenamiento y prediccion (Regresion Lineal)

In [61]:
scaler = StandardScaler()
clf = LinearRegression()

pipe = Pipeline([
    ('scaler', scaler),
    ('regressor', clf)
])

pipe.fit(X_trai, Y_trai)
Y_pred = pipe.predict(X_test)

mean_squared_error(Y_pred, Y_test)

20.93860578420929

### Preprocesamiento, entrenamiento y prediccion (Regresion Arbol de Decision)

In [65]:
scaler = StandardScaler()
clf = DecisionTreeRegressor(random_state=0)

pipe = Pipeline([
    ('scaler', scaler),
    ('regressor', clf)
])

pipe.fit(X_trai, Y_trai)
Y_pred = pipe.predict(X_test)

mean_squared_error(Y_pred, Y_test)

21.397251201671892

### Preprocesamiento, entrenamiento y prediccion (Regresion MLP Network)

In [73]:
scaler = StandardScaler()
clf = MLPRegressor(hidden_layer_sizes=(10,8,10), activation='relu', solver='adam', max_iter=500)

pipe = Pipeline([
    ('scaler', scaler),
    ('regressor', clf)
])

pipe.fit(X_trai, Y_trai)
Y_pred = pipe.predict(X_test)

mean_squared_error(Y_pred, Y_test)

19.635162524147567